In [9]:
import pandas as pd
import requests
from tqdm.auto import tqdm
from pprint import pprint
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics
from rectools.model_selection import TimeRangeSplitter
from rectools.models.popular import PopularModel
import numpy as np

from userknn import UserKnn

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Датасет KION

In [2]:
# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
# req = requests.get(url, stream=True)

# with open('kion_train.zip', "wb") as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='Downloading the kion dataset...',
#                         total=total_size_in_bytes,
#                         unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

In [3]:
# !unzip kion_train.zip -x '__MACOSX/*'

In [4]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True)
print(interactions_df.shape)
# will cast types and save new pd.DataFrame inside in Interactions.df
interactions = Interactions(interactions_df)

# ! если хотите быстро прогнать этот ноутбук - раскомментируйте эту строку - она уменьшает данные
# interactions = Interactions(interactions_df.sample(frac=0.01))

interactions.df.head()

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


## Задаем фолды для кросс-валидации

In [5]:
N_SPLITS = 3
TEST_SIZE = '7D'

In [6]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [7]:
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-08-02 00:00:00', freq='7D'),
  Timestamp('2021-08-09 00:00:00', freq='7D')),
 (Timestamp('2021-08-09 00:00:00', freq='7D'),
  Timestamp('2021-08-16 00:00:00', freq='7D')),
 (Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

In [8]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10),
}

# few simple models to compare
models = {
    'cosine_userknn_20': CosineRecommender(),  # implicit
    'tfidf_userknn_20': TFIDFRecommender(),
    'bm25_userknn_20': BM25Recommender(),
    'cosine_userknn_10': CosineRecommender(K=10),
    'tfidf_userknn_10': TFIDFRecommender(K=10),
    'bm25_userknn_10': BM25Recommender(K=10),
}

# CV

In [25]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)
    
        recos = userknn_model.predict(df_test)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)

In [10]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,map@10,novelty
0,0,cosine_userknn_20,0.004246,7.476357
1,0,tfidf_userknn_20,0.006773,7.573736
2,0,bm25_userknn_20,0.002784,9.199335
3,0,cosine_userknn_10,0.004200,6.937947
4,0,tfidf_userknn_10,0.005957,7.378542
5,0,bm25_userknn_10,0.003256,8.699078
6,1,cosine_userknn_20,0.003785,7.531878
7,1,tfidf_userknn_20,0.006279,7.631293
8,1,bm25_userknn_20,0.002679,9.285580
9,1,cosine_userknn_10,0.003900,6.983614


In [11]:
df_metrics.groupby('model').mean()[metrics.keys()]

,map@10,novelty
model,,
bm25_userknn_10,0.003164,8.776933
bm25_userknn_20,0.002699,9.283169
cosine_userknn_10,0.003958,6.992581
cosine_userknn_20,0.003905,7.540823
tfidf_userknn_10,0.005659,7.433605
tfidf_userknn_20,0.006334,7.638598


# Train full model for service

In [10]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True)
print(interactions_df.shape)
# will cast types and save new pd.DataFrame inside in Interactions.df
interactions = Interactions(interactions_df)

# ! если хотите быстро прогнать этот ноутбук - раскомментируйте эту строку - она уменьшает данные
# interactions = Interactions(interactions_df.sample(frac=0.01))

interactions.df.head()

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [11]:
pop_model = PopularModel()
tfidf_model = UserKnn(model=TFIDFRecommender(), N_users=50)

df_train = interactions.df
dataset = Dataset.construct(
    interactions_df=df_train,
    user_features_df=None,
    item_features_df=None,
)
catalog = df_train[Columns.Item].unique()

In [12]:
tfidf_model.fit(df_train)
pop_model.fit(dataset)

  0%|          | 0/962179 [00:00<?, ?it/s]

In [13]:
tfidf_recos = tfidf_model.predict(df_train)
pop_recos = pop_model.recommend(
    users=np.unique(df_train[Columns.User]),
    dataset=dataset,
    k=10,
    filter_viewed=True
    )

In [14]:
tfidf_recos.head()

,user_id,item_id,score,rank
2,1097557,3182,6.137841,1
0,1097557,4151,4.111983,2
1,1097557,15297,3.379502,3
3,1097557,10440,3.029132,4
4,1097556,12812,8.013524,1


In [15]:
pop_recos.head()

,user_id,item_id,score,rank
0,0,10440,202457.0,1
1,0,13865,122119.0,2
2,0,4151,91167.0,3
3,0,3734,74803.0,4
4,0,2657,68581.0,5


In [21]:
pop_recos = pop_recos.groupby("user_id")["item_id"].agg(list).to_dict()
tfidf_recos = tfidf_recos.groupby("user_id")["item_id"].agg(list).to_dict()

In [22]:
for key in tfidf_recos.keys():
    if len(tfidf_recos[key]) < 10:
        while len(tfidf_recos[key]) < 10:
            cur_item = pop_recos[key].pop(0)
            if cur_item not in tfidf_recos[key]:
                tfidf_recos[key].append(cur_item)

In [24]:
import json

with open("tfidf_20_model.json", "w") as f:
    json.dump(tfidf_recos, f)